In [1]:
import os

In [2]:
%pwd

'/Users/tajbirhasanshuvo/academicfiles/chest-cancer-classification/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/tajbirhasanshuvo/academicfiles/chest-cancer-classification'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/tajshuvo75/chest-cancer-classification.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="tajshuvo75"
os.environ["MLFLOW_TRACKING_PASSWORD"]="52d418994430c9adfa7397593b8d71a090c99994"

In [6]:
import tensorflow as tf

In [7]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

2026-02-04 23:36:18.496561: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2026-02-04 23:36:18.496584: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2026-02-04 23:36:18.496591: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.92 GB
2026-02-04 23:36:18.496624: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2026-02-04 23:36:18.496638: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from chest_cancer_classification.constants import *
from chest_cancer_classification.utils.common import read_yaml, create_directories

In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/tajshuvo75/chest-cancer-classification.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config




In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.tensorflow
from urllib.parse import urlparse
from chest_cancer_classification.utils.common import save_json


In [12]:
import tensorflow.keras as keras
keras.__version__ = "2.13.0"


In [ ]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_tracking_uri(self.config.mlflow_uri)

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )

            mlflow.tensorflow.log_model(
                self.model,
                artifact_path="model"
            )




In [14]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2026-02-04 23:36:19,914: INFO: common: yaml file: config/config.yaml loaded successfully]
[2026-02-04 23:36:19,915: INFO: common: yaml file: params.yaml loaded successfully]
[2026-02-04 23:36:19,916: INFO: common: created directory at: artifacts]
Found 102 images belonging to 2 classes.


2026-02-04 23:36:20.540861: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


7/7 [==============================] - 2s 123ms/step - loss: 0.0539 - accuracy: 0.9902
[2026-02-04 23:36:22,045: INFO: common: json file saved at: scores.json]


2026/02/04 23:36:24 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /var/folders/sr/v_bsvj7j3lgg41xztlkdg5zh0000gn/T/tmpojo6xp37/model/data/model/assets
[2026-02-04 23:36:24,969: INFO: builder_impl: Assets written to: /var/folders/sr/v_bsvj7j3lgg41xztlkdg5zh0000gn/T/tmpojo6xp37/model/data/model/assets]


2026/02/04 23:36:29 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /var/folders/sr/v_bsvj7j3lgg41xztlkdg5zh0000gn/T/tmpojo6xp37/model, flavor: tensorflow), fall back to return ['tensorflow==2.13.0']. Set logging level to DEBUG to see the full traceback.
